In [1]:
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import kneighbors_graph
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
from anndata import AnnData
import pandas as pd
import numpy as np
import seaborn as sns
import colorcet as cc
import scanpy as sc
import igraph as ig
import colorcet as cc
import leidenalg
import umap
import copy
import os

from functions import map_scatter, glasbey

In [3]:
path9 = "G:/.shortcut-targets-by-id/1AK4zOnr51LD4uof7We2mDh36g2PGNA7b/Mayo Clinic Florida - Justin Nguyen/Data/`per image cell measurements/Slide9-CellMeasurements.csv"
path10 = "G:/.shortcut-targets-by-id/1AK4zOnr51LD4uof7We2mDh36g2PGNA7b/Mayo Clinic Florida - Justin Nguyen/Data/`per image cell measurements/Slide10-CellMeasurements.csv"
path11 = "G:/.shortcut-targets-by-id/1AK4zOnr51LD4uof7We2mDh36g2PGNA7b/Mayo Clinic Florida - Justin Nguyen/Data/`per image cell measurements/Slide11-CellMeasurements.csv"
path12 = "G:/.shortcut-targets-by-id/1AK4zOnr51LD4uof7We2mDh36g2PGNA7b/Mayo Clinic Florida - Justin Nguyen/Data/`per image cell measurements/Slide12-CellMeasurements.csv"
image_paths = [path9, path10, path11, path12]

In [4]:
df_lst = []
inc_cols = ['Image', 'Object ID', 'Name', 'Classification', 'Parent', 'Centroid X µm', 'Centroid Y µm', 'DAPI: Cell: Mean', 'CD44: Cell: Mean', 'HLA-DR: Cell: Mean', 'CD4: Cell: Mean', 'IFNG: Cell: Mean', 'Ki67: Cell: Mean', 'CD107a: Cell: Mean', 'CD45: Cell: Mean', 'CD20: Cell: Mean', 'CD40: Cell: Mean', 'CD8: Cell: Mean', 'Pan-Cytokeratin: Cell: Mean', 'CD68: Cell: Mean', 'HLA-A: Cell: Mean', 'CD79a: Cell: Mean', 'CD45RO: Cell: Mean', 'CD21: Cell: Mean', 'CD11c: Cell: Mean', 'HLA-E: Cell: Mean', 'IDO1: Cell: Mean', 'CD14: Cell: Mean', 'CD56: Cell: Mean', 'VISTA: Cell: Mean', 'FOXP3: Cell: Mean', 'Granzyme B: Cell: Mean', 'PCNA: Cell: Mean', 'T-bet/TBX21: Cell: Mean', 'PD-L1: Cell: Mean', 'TOX: Cell: Mean', 'PD-1: Cell: Mean', 'CD38: Cell: Mean', 'ICOS: Cell: Mean', 'CD39: Cell: Mean', 'LAG3: Cell: Mean', 'TCF-1: Cell: Mean', 'CD3e: Cell: Mean']
for path in image_paths:
    df = pd.read_csv(path, sep='\t', usecols=inc_cols)
    df_lst.append(df)
df_lst[0].head()

,Image,Object ID,Name,Classification,Parent,Centroid X µm,Centroid Y µm,DAPI: Cell: Mean,CD44: Cell: Mean,HLA-DR: Cell: Mean,...,T-bet/TBX21: Cell: Mean,PD-L1: Cell: Mean,TOX: Cell: Mean,PD-1: Cell: Mean,CD38: Cell: Mean,ICOS: Cell: Mean,CD39: Cell: Mean,LAG3: Cell: Mean,TCF-1: Cell: Mean,CD3e: Cell: Mean
0,Slide9_Scan1.ome.tif,3c467323-15ef-44ac-a660-3c2db2b3422c,NaN,NaN,Annotation (Slide 9 A1),7389.8,3296.6,118.6025,0.3725,0.8331,...,3.3991,0.0504,0.0017,0.0127,0.0000,0.0671,0.0732,0.7295,0.2312,0.1236
1,Slide9_Scan1.ome.tif,fbd8d545-c663-4aac-b5c8-7f3e3af3a95e,NaN,NaN,Annotation (Slide 9 A1),7544.4,3300.9,129.0983,2.1161,0.6948,...,2.3923,0.6179,0.0000,0.0200,0.0227,0.2731,0.0200,1.8572,2.1294,0.5107
2,Slide9_Scan1.ome.tif,82adeef0-2804-4bce-9e1f-12b11bf9fe1c,NaN,CD68,Annotation (Slide 9 A1),7556.3,3302.0,129.9955,2.2608,1.7795,...,1.7602,0.8494,0.0074,0.0938,0.2455,1.5187,0.0864,2.7313,4.4636,0.7148
3,Slide9_Scan1.ome.tif,92bea315-98b1-44fb-bece-d79a4aeb609e,NaN,NaN,Annotation (Slide 9 A1),7384.1,3304.0,107.7341,0.7630,0.0993,...,0.2156,0.0304,0.0000,0.0822,0.0326,0.0933,0.2444,0.5133,0.2815,0.2067
4,Slide9_Scan1.ome.tif,14e1d90e-4615-4e68-807f-fb871b565e69,NaN,NaN,Annotation (Slide 9 A1),7522.1,3304.4,90.3814,1.1816,0.9844,...,1.9307,0.3452,0.0537,0.0449,0.1779,0.3670,0.2591,1.2335,1.1629,0.5562


In [5]:
df = pd.concat(df_lst, ignore_index=True)
print(df['Parent'].unique())

['Annotation (Slide 9 A1)' 'Annotation (Slide 9 B2)'
 'Annotation (Slide 9 A1-1)' 'Annotation (Slide 10 C1-1)'
 'Annotation (Slide 10 D1)' 'Annotation (Slide 10 C1)'
 'Annotation (Slide 11 B1)' 'Annotation (Slide 11 A1)'
 'Annotation (Slide 11 A1-1)' 'Annotation (Slide 12 E1)'
 'Annotation (Slide 12 C1)' 'Annotation (Slide 12 C1-1)']


In [6]:
nan_counts = df.isna().sum()
print(nan_counts)

Image                                0
Object ID                            0
Name                           4748619
Classification                  921719
Parent                               0
Centroid X µm                        0
Centroid Y µm                        0
DAPI: Cell: Mean                     0
CD44: Cell: Mean                     0
HLA-DR: Cell: Mean                   0
CD4: Cell: Mean                      0
IFNG: Cell: Mean                     0
Ki67: Cell: Mean                     0
CD107a: Cell: Mean                   0
CD45: Cell: Mean                     0
CD20: Cell: Mean                     0
CD40: Cell: Mean                     0
CD8: Cell: Mean                      0
Pan-Cytokeratin: Cell: Mean          0
CD68: Cell: Mean                     0
HLA-A: Cell: Mean                    0
CD79a: Cell: Mean                    0
CD45RO: Cell: Mean                   0
CD21: Cell: Mean                     0
CD11c: Cell: Mean                    0
HLA-E: Cell: Mean        

In [7]:
df = df.drop(columns=['Name'])
# fill NaN classification with 'Other/NA'
df['Classification'] = df['Classification'].fillna('Other/NA')
print(df.isna().sum())

Image                          0
Object ID                      0
Classification                 0
Parent                         0
Centroid X µm                  0
Centroid Y µm                  0
DAPI: Cell: Mean               0
CD44: Cell: Mean               0
HLA-DR: Cell: Mean             0
CD4: Cell: Mean                0
IFNG: Cell: Mean               0
Ki67: Cell: Mean               0
CD107a: Cell: Mean             0
CD45: Cell: Mean               0
CD20: Cell: Mean               0
CD40: Cell: Mean               0
CD8: Cell: Mean                0
Pan-Cytokeratin: Cell: Mean    0
CD68: Cell: Mean               0
HLA-A: Cell: Mean              0
CD79a: Cell: Mean              0
CD45RO: Cell: Mean             0
CD21: Cell: Mean               0
CD11c: Cell: Mean              0
HLA-E: Cell: Mean              0
IDO1: Cell: Mean               0
CD14: Cell: Mean               0
CD56: Cell: Mean               0
VISTA: Cell: Mean              0
FOXP3: Cell: Mean              0
Granzyme B

In [8]:
# rename columns for readability
df.columns = [col.replace(': Cell: Mean', '') for col in df.columns]
print(df.columns)

Index(['Image', 'Object ID', 'Classification', 'Parent', 'Centroid X µm',
       'Centroid Y µm', 'DAPI', 'CD44', 'HLA-DR', 'CD4', 'IFNG', 'Ki67',
       'CD107a', 'CD45', 'CD20', 'CD40', 'CD8', 'Pan-Cytokeratin', 'CD68',
       'HLA-A', 'CD79a', 'CD45RO', 'CD21', 'CD11c', 'HLA-E', 'IDO1', 'CD14',
       'CD56', 'VISTA', 'FOXP3', 'Granzyme B', 'PCNA', 'T-bet/TBX21', 'PD-L1',
       'TOX', 'PD-1', 'CD38', 'ICOS', 'CD39', 'LAG3', 'TCF-1', 'CD3e'],
      dtype='object')


In [9]:
metadata = ['Image', 'Object ID', 'Classification', 'Parent', 'Centroid X µm', 'Centroid Y µm']
markers = [col for col in df.columns if col not in metadata]
print(markers)

['DAPI', 'CD44', 'HLA-DR', 'CD4', 'IFNG', 'Ki67', 'CD107a', 'CD45', 'CD20', 'CD40', 'CD8', 'Pan-Cytokeratin', 'CD68', 'HLA-A', 'CD79a', 'CD45RO', 'CD21', 'CD11c', 'HLA-E', 'IDO1', 'CD14', 'CD56', 'VISTA', 'FOXP3', 'Granzyme B', 'PCNA', 'T-bet/TBX21', 'PD-L1', 'TOX', 'PD-1', 'CD38', 'ICOS', 'CD39', 'LAG3', 'TCF-1', 'CD3e']


In [10]:
# remove the Annotation word in Parent column for readability
df['Parent'] = df['Parent'].str.replace('Annotation (', '', regex=False).str.replace(')', '', regex=False)
print(df['Parent'].unique())

['Slide 9 A1' 'Slide 9 B2' 'Slide 9 A1-1' 'Slide 10 C1-1' 'Slide 10 D1'
 'Slide 10 C1' 'Slide 11 B1' 'Slide 11 A1' 'Slide 11 A1-1' 'Slide 12 E1'
 'Slide 12 C1' 'Slide 12 C1-1']


In [11]:
df['Slide'] = df['Parent'].str.extract(r'(Slide \d+)')
df.head()

,Image,Object ID,Classification,Parent,Centroid X µm,Centroid Y µm,DAPI,CD44,HLA-DR,CD4,...,PD-L1,TOX,PD-1,CD38,ICOS,CD39,LAG3,TCF-1,CD3e,Slide
0,Slide9_Scan1.ome.tif,3c467323-15ef-44ac-a660-3c2db2b3422c,Other/NA,Slide 9 A1,7389.8,3296.6,118.6025,0.3725,0.8331,0.5854,...,0.0504,0.0017,0.0127,0.0000,0.0671,0.0732,0.7295,0.2312,0.1236,Slide 9
1,Slide9_Scan1.ome.tif,fbd8d545-c663-4aac-b5c8-7f3e3af3a95e,Other/NA,Slide 9 A1,7544.4,3300.9,129.0983,2.1161,0.6948,1.3029,...,0.6179,0.0000,0.0200,0.0227,0.2731,0.0200,1.8572,2.1294,0.5107,Slide 9
2,Slide9_Scan1.ome.tif,82adeef0-2804-4bce-9e1f-12b11bf9fe1c,CD68,Slide 9 A1,7556.3,3302.0,129.9955,2.2608,1.7795,4.6847,...,0.8494,0.0074,0.0938,0.2455,1.5187,0.0864,2.7313,4.4636,0.7148,Slide 9
3,Slide9_Scan1.ome.tif,92bea315-98b1-44fb-bece-d79a4aeb609e,Other/NA,Slide 9 A1,7384.1,3304.0,107.7341,0.7630,0.0993,0.1681,...,0.0304,0.0000,0.0822,0.0326,0.0933,0.2444,0.5133,0.2815,0.2067,Slide 9
4,Slide9_Scan1.ome.tif,14e1d90e-4615-4e68-807f-fb871b565e69,Other/NA,Slide 9 A1,7522.1,3304.4,90.3814,1.1816,0.9844,2.3521,...,0.3452,0.0537,0.0449,0.1779,0.3670,0.2591,1.2335,1.1629,0.5562,Slide 9


In [12]:
# log normalization and scaling function
# args has the columns to include for that dataset
def log_norm_scale(group, *args):
    include = args[0]
    # reset index to avoid duplicated indices
    group_reset = group.reset_index(level='Slide', drop=True)
    # apply log normalization
    group_reset[include] = np.log(group_reset[include] + 1)
    # apply standard scaling
    scaler = StandardScaler()
    group_reset[include] = scaler.fit_transform(group_reset[include])
    return group_reset

In [13]:
# apply function to each patient
df = df.set_index('Slide')
df = df.groupby(level='Slide').apply(log_norm_scale, markers)
df.head()

Image                             Object ID  \
Slide                                                                     
Slide 10 0  Slide10_Scan1.ome.tif  f5505ea9-ddff-4fea-90dd-a425402a544d   
         1  Slide10_Scan1.ome.tif  4b313e44-f97c-43e8-99a7-31b9ab2a45c0   
         2  Slide10_Scan1.ome.tif  e0673808-3a80-4490-b565-2fa9b3425738   
         3  Slide10_Scan1.ome.tif  e2036b6c-b8ba-42fc-b05c-1e0c8c853192   
         4  Slide10_Scan1.ome.tif  5afebba8-c97a-4d69-b9e2-395d49cebd90   

           Classification         Parent  Centroid X µm  Centroid Y µm  \
Slide                                                                    
Slide 10 0       Other/NA  Slide 10 C1-1         4004.2         1674.9   
         1       Other/NA  Slide 10 C1-1         4055.9         1675.2   
         2           CD56  Slide 10 C1-1         4070.8         1675.8   
         3       Other/NA  Slide 10 C1-1         3904.7         1677.1   
         4       Other/NA  Slide 10 C1-1         3921.0         1676.8   

                DAPI      CD44    HLA-DR       CD4  ...  T-bet/TBX21  \
Slide                                               ...                
Slide 10 0 -0.443124 -1.278166 -0.611227 -0.836912  ...     0.569947   
         1 -1.365461 -1.426814 -0.966250 -1.498882  ...     0.438516   
         2 -0.981877 -1.412553 -0.762933 -1.413784  ...     0.398702   
         3 -0.554083 -1.372611  0.236735 -0.793350  ...     0.503248   
         4 -0.943903 -1.384704 -0.198105 -1.196011  ...     0.525127   

               PD-L1       TOX      PD-1      CD38      ICOS      CD39  \
Slide                                                                    
Slide 10 0 -1.092399 -0.512276 -0.536722 -0.767958 -0.900027 -0.530450   
         1 -1.100433 -0.512276 -0.531385 -0.767958 -0.945659 -0.530450   
         2 -1.019284 -0.509256 -0.503575 -0.763376 -0.841767 -0.503103   
         3  0.066361 -0.512276 -0.473942 -0.766839 -0.938628 -0.462030   
         4 -1.100433 -0.512276 -0.536722 -0.767958 -0.817616 -0.530450   

                LAG3     TCF-1      CD3e  
Slide                                     
Slide 10 0 -1.661986 -1.439516 -0.895539  
         1 -1.541051 -1.406103 -1.080230  
         2 -1.088439 -0.997410 -1.053522  
         3 -1.421700 -1.354550 -1.077320  
         4 -1.534252 -1.337009 -1.067024  

[5 rows x 42 columns]

In [15]:
df = df.reset_index(drop=True)
df.head()

,Image,Object ID,Classification,Parent,Centroid X µm,Centroid Y µm,DAPI,CD44,HLA-DR,CD4,...,T-bet/TBX21,PD-L1,TOX,PD-1,CD38,ICOS,CD39,LAG3,TCF-1,CD3e
0,Slide10_Scan1.ome.tif,f5505ea9-ddff-4fea-90dd-a425402a544d,Other/NA,Slide 10 C1-1,4004.2,1674.9,-0.443124,-1.278166,-0.611227,-0.836912,...,0.569947,-1.092399,-0.512276,-0.536722,-0.767958,-0.900027,-0.530450,-1.661986,-1.439516,-0.895539
1,Slide10_Scan1.ome.tif,4b313e44-f97c-43e8-99a7-31b9ab2a45c0,Other/NA,Slide 10 C1-1,4055.9,1675.2,-1.365461,-1.426814,-0.966250,-1.498882,...,0.438516,-1.100433,-0.512276,-0.531385,-0.767958,-0.945659,-0.530450,-1.541051,-1.406103,-1.080230
2,Slide10_Scan1.ome.tif,e0673808-3a80-4490-b565-2fa9b3425738,CD56,Slide 10 C1-1,4070.8,1675.8,-0.981877,-1.412553,-0.762933,-1.413784,...,0.398702,-1.019284,-0.509256,-0.503575,-0.763376,-0.841767,-0.503103,-1.088439,-0.997410,-1.053522
3,Slide10_Scan1.ome.tif,e2036b6c-b8ba-42fc-b05c-1e0c8c853192,Other/NA,Slide 10 C1-1,3904.7,1677.1,-0.554083,-1.372611,0.236735,-0.793350,...,0.503248,0.066361,-0.512276,-0.473942,-0.766839,-0.938628,-0.462030,-1.421700,-1.354550,-1.077320
4,Slide10_Scan1.ome.tif,5afebba8-c97a-4d69-b9e2-395d49cebd90,Other/NA,Slide 10 C1-1,3921.0,1676.8,-0.943903,-1.384704,-0.198105,-1.196011,...,0.525127,-1.100433,-0.512276,-0.536722,-0.767958,-0.817616,-0.530450,-1.534252,-1.337009,-1.067024


In [16]:
sp = os.path.join('..', 'outputs', 'clean.csv')
df.to_csv(sp, index=False)